In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import col, concat_ws, lag

# Initialize a Spark session
spark = SparkSession.builder.appName("FlightTurns").getOrCreate()

# You can replace this with your actual DataFrame creation code
df = spark.read.csv("file.csv", header=True, sep=";")

# Sort the DataFrame by aircraft registration code and departure timestamp
sorted_df = df.orderBy("acft_regs_cde", "actl_dep_lcl_tms")

# Calculate the time difference between consecutive rows for the same aircraft
window_spec = Window.partitionBy("acft_regs_cde").orderBy("actl_dep_lcl_tms")
sorted_df = sorted_df.withColumn("time_diff", col("actl_dep_lcl_tms").cast("long") - lag("actl_arr_lcl_tms").over(window_spec))

# Create a new column with a string of flights based on the turn of the aircraft
sorted_df = sorted_df.withColumn("next_flights", concat_ws(", ", "id", "orig", "dest", "actl_dep_lcl_tms", "actl_arr_lcl_tms", "flight_num"))

#task 1
# Show the resulting DataFrame
sorted_df.show(truncate=False)

# Initialize a Spark session
spark = SparkSession.builder.appName("AirportFlights").getOrCreate()

# You can replace this with your actual DataFrame creation code
 df = spark.read.csv("file.csv", header=True, sep=";")

# Define the time format
time_format = "yyyy-MM-dd'T'HH:mm:ss"

# Create a window specification for the time intervals
window_spec = window(col("actl_dep_lcl_tms"), "2 hours", "15 minutes")

# Group the data by airport codes (origin and destination) and the time window
airport_counts = df.groupBy("orig", window_spec).count().orderBy("window")

#task 2
# Show the resulting DataFrame
airport_counts.show(truncate=False)

# Stop the Spark session
spark.stop()


IndentationError: ignored